# VarianceComponentModels.jl

[`VarianceComponentModels.jl`](https://github.com/OpenMendel/VarianceComponentModels.jl/) is a package that resides in [OpenMendel](https://github.com/OpenMendel) ecosystem. It implements computation routines for fitting and testing variance component model of form 

$$\text{vec}(Y) \sim \text{Normal}(XB, \Sigma_1 \otimes V_1 + \cdots + \Sigma_m \otimes V_m)$$

where $\otimes$ is the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product). 




### Package Features 
* Maximum likelihood estimation (MLE) and restricted maximum likelihood estimation (REML) of mean parameters B and variance component parameters Σ
* Allow constrains in the mean parameters B
* Choice of optimization algorithms: [Fisher scoring](https://books.google.com/books?id=QYqeYTftPNwC&lpg=PP1&pg=PA142#v=onepage&q&f=false) and [minorization-maximization algorithm](http://hua-zhou.github.io/media/pdf/ZhouHuZhouLange19VCMM.pdf)
* [Heritability Analysis](https://openmendel.github.io/VarianceComponentModels.jl/latest/man/heritability/#Heritability-Analysis-1) in genetics

### Installation

This package requires Julia v0.7.0 or later, which can be obtained from https://julialang.org/downloads/ or by building Julia from the sources in the https://github.com/JuliaLang/julia repository.

The package has not yet been registered and must be installed using the repository location. Start julia and use the ] key to switch to the package manager REPL

```julia
(v1.3) pkg> add https://github.com/OpenMendel/VarianceComponentModels.jl.git#Julia-0.7
```

Use the backspace key to return to the Julia REPL.

### Outline
* [MLE and REML](#MLE-and-REML) 
* [Heritability Analysis](#Heritability-Analysis)

## MLE and REML 

Machine Information:

In [8]:
versioninfo()

Julia Version 1.3.1
Commit 2d5741174c (2019-12-30 21:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


### Demo data 

For demonstration, we generate a random data set. 

In [5]:
# generate data from a d-variate response variane component model
using Random, LinearAlgebra
Random.seed!(123)
n = 1000   # no. observations
d = 2      # dimension of responses
m = 2      # no. variance components
p = 2      # no. covariates
# n-by-p design matrix
X = randn(n, p)
# p-by-d mean component regression coefficient
B = ones(p, d)  
# a tuple of m covariance matrices
V = ntuple(x -> zeros(n, n), m) 
for i = 1:m-1
  Vi = randn(n, 50)
  copyto!(V[i], Vi * Vi')
end
copyto!(V[m], Matrix(I, n, n)) # last covarianec matrix is idendity
# a tuple of m d-by-d variance component parameters
Σ = ntuple(x -> zeros(d, d), m) 
for i in 1:m
  Σi = randn(d, d)
  copyto!(Σ[i], Σi' * Σi)
end
# form overall nd-by-nd covariance matrix Ω
Ω = zeros(n * d, n * d)
for i = 1:m
  Ω += kron(Σ[i], V[i])
end
Ωchol = cholesky(Ω)
# n-by-d responses
Y = X * B + reshape(Ωchol.L * randn(n*d), n, d);

### Maximum likelihood estimation (MLE)

To find the MLE of parameters $(B,\Sigma_1,\ldots,\Sigma_m)$, we take 3 steps: 

**Step 1 (Construct data)**. Construct an instance of `VarianceComponentVariate`, which consists of fields 

* `Y`: $n$-by-$d$ responses
* `X`: $n$-by-$p$ covariate matrix 
* `V=(V[1],...,V[m])`: a tuple of $n$-by-$n$ covariance matrices. The last covariance matrix must be positive definite and usually is the identity matrix.

In [10]:
using VarianceComponentModels
vcdata = VarianceComponentVariate(Y, X, V)
fieldnames(typeof(vcdata))

(:Y, :X, :V)

In the absence of covariates $X$, we can simply initialize by `vcdata = VarianceComponentVariate(Y, V)`.

**Step 2 (Construct a model)**. Construct an instance of `VarianceComponentModel`, which consists of fields

* `B`: $n$-by-$p$ mean regression coefficients
* `Σ=(Σ[1],...,Σ[m])`: variane component parameters respectively.

When constructed from a VarianceComponentVariate instance, the mean parameters $B$ are initialized to be zero and the tuple of variance component parameters $\Sigma$ to be `(eye(d),...,eye(d))`.

In [11]:
vcmodel = VarianceComponentModel(vcdata)
fieldnames(typeof(vcmodel))

(:B, :Σ, :A, :sense, :b, :lb, :ub)

In [12]:
vcmodel

VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.0 0.0; 0.0 0.0], ([1.0 0.0; 0.0 1.0], [1.0 0.0; 0.0 1.0]), Array{Float64}(undef,0,4), Char[], Float64[], -Inf, Inf)

The remaining fields `A`, `sense`, `b`, `lb`, `ub` specify (optional) constraints on the mean parameters `B`:

$A * \text{vec}(B) =(\text{or } \ge \text{or } \le) b$

$lb \le \text{vec}(B) \le ub$

`A` is an constraint matrix with $pd$ columns, `sense` is a vector of charaters taking values `'<'`, `'='` or `'>'`, and `lb` and `ub` are the lower and upper bounds for `vec(B)`. By default, `A`, `sense`, `b` are empty, `lb` is `-Inf`, and `ub` is `Inf`. If any constraits are non-trivial, final estimates of `B` are enforced to satisfy them.

When a better initial guess is available, we can initialize by calling `vcmodel=VarianceComponentModel(B0, Σ0)` directly.

**Step 3 (Fit model)**. Call optmization routine `fit_mle!`. The keyword `algo` dictates the optimization algorithm: `:MM` (minorization-maximization algorithm) or `:FS` (Fisher scoring algorithm).

In [13]:
vcmodel_mle = deepcopy(vcmodel)
@time logl, vcmodel_mle, Σse, Σcov, Bse, Bcov = fit_mle!(vcmodel_mle, vcdata; algo = :MM);


     MM Algorithm
  Iter      Objective  
--------  -------------
       0  -6.253551e+03

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

       1  -3.881454e+03
       2  -3.853179e+03
       3  -3.846525e+03
       4  -3.844906e+03
       5  -3.844506e+03
       6  -3.844406e+03
       7  -3.844381e+03
       8  -3.844375e+03
       9  -3.844374e+03
      10  -3.844373e+03

  5.285482 seconds (11.44 M allocations: 570.981 MiB, 3.06% gc time)


The output of `fit_mle!` contains

* final log-likelihood

In [18]:
logl

-3844.3731814180883

* fitted model

In [19]:
fieldnames(typeof(vcmodel_mle))


(:B, :Σ, :A, :sense, :b, :lb, :ub)

In [20]:
vcmodel_mle

VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([1.0919960341406953 1.0472703873788904; 0.9553455716716395 1.016315638674718], ([0.3806371846890906 -0.30546523211690463; -0.30546523211690463 4.519380324112873], [1.8400909904775102 0.26556859919650166; 0.26556859919650166 2.172752980034015]), Array{Float64}(undef,0,4), Char[], Float64[], -Inf, Inf)

* standard errors of the estimated varianec component parameters

In [17]:
Σse

([0.07651361924362647 0.2630470127784574; 0.2630470127784576 0.9043321868733178], [0.08442916131150752 0.09174407350754034; 0.09174407350754071 0.09969274380846048])

* covariance matrix of the variance component parameters estimates

In [16]:
Σcov

8×8 Array{Float64,2}:
  0.00585433  -0.00467019  -0.00467019  …  -1.07903e-6   -1.557e-7   
 -0.00467019   0.0691937    0.00372555     -1.557e-7     -1.27444e-6 
 -0.00467019   0.00372555   0.0691937      -8.83212e-6   -1.27444e-6 
  0.00372555  -0.055198    -0.055198       -1.27444e-6   -1.04316e-5 
 -7.4779e-6   -1.07903e-6  -1.07903e-6      0.00102878    0.000148477
 -1.07903e-6  -8.83212e-6  -1.557e-7    …   0.000148477   0.00121477 
 -1.07903e-6  -1.557e-7    -8.83212e-6      0.00841698    0.00121477 
 -1.557e-7    -1.27444e-6  -1.27444e-6      0.00121477    0.00993864 

* standard errors of the estimated mean parameters

In [15]:
Bse

2×2 Array{Float64,2}:
 0.0425562  0.0483834
 0.0430596  0.0492809

* covariance matrix of the mean parameter estimates

In [14]:
Bcov

4×4 Array{Float64,2}:
  0.00181103   -1.96485e-5    0.000243441  -4.38252e-6 
 -1.96485e-5    0.00185413   -4.38252e-6    0.000246407
  0.000243441  -4.38252e-6    0.00234096   -5.73331e-6 
 -4.38252e-6    0.000246407  -5.73331e-6    0.00242861 

### Restricted maximum likelihood estimation (REML)

[REML (restricted maximum likelihood estimation)](https://en.wikipedia.org/wiki/Restricted_maximum_likelihood) is a popular alternative to the MLE. To find the REML of a variane component model, we replace the above step 3 by

**Step 3**. Call optmization routine `fit_reml!`.

In [21]:
vcmodel_reml = deepcopy(vcmodel)
@time logl, vcmodel_reml, Σse, Σcov, Bse, Bcov = fit_reml!(vcmodel_reml, vcdata; algo = :MM);


     MM Algorithm
  Iter      Objective  
--------  -------------
       0  -4.215053e+03
       1  -3.925799e+03
       2  -3.865114e+03
       3  -3.851105e+03
       4  -3.847732e+03
       5  -3.846903e+03
       6  -3.846698e+03
       7  -3.846647e+03
       8  -3.846634e+03
       9  -3.846631e+03
      10  -3.846630e+03

  0.574157 seconds (179.03 k allocations: 71.507 MiB, 13.59% gc time)


The output of fit_reml! contains

* the final log-likelihood at REML estimate

In [23]:
logl

-3844.3777179025046

* REML estimates

In [24]:
fieldnames(typeof(vcmodel_reml))

(:B, :Σ, :A, :sense, :b, :lb, :ub)

In [25]:
vcmodel_reml

VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([1.0919959427065358 1.0472704732594296; 0.9553454519764125 1.0163157746706337], ([0.38059386475358836 -0.30548508022759785; -0.30548508022759785 4.51994275993852], [1.8428522689608946 0.26196264194000846; 0.26196264194000846 2.1784163819904427]), Array{Float64}(undef,0,4), Char[], Float64[], -Inf, Inf)

* standard errors of the estimated variance component parameters

In [26]:
Σse

([0.07650553461473508 0.26304964665017927; 0.2630496466501792 0.904445863017746], [0.08455585744252603 0.09193246406052916; 0.09193246406052925 0.09995259851207343])

* covariance matrix of the variance component parameters estimates

In [27]:
Σcov

8×8 Array{Float64,2}:
  0.0058531   -0.00467005  -0.00467005  …  -1.06597e-6   -1.51499e-7 
 -0.00467005   0.0691951    0.00372613     -1.51499e-7   -1.26041e-6 
 -0.00467005   0.00372613   0.0691951      -8.86843e-6   -1.26041e-6 
  0.00372613  -0.0552092   -0.0552092      -1.26041e-6   -1.0486e-5  
 -7.50035e-6  -1.06597e-6  -1.06597e-6      0.00101633    0.000144472
 -1.06597e-6  -8.86843e-6  -1.51499e-7  …   0.000144472   0.0012014  
 -1.06597e-6  -1.51499e-7  -8.86843e-6      0.00845158    0.0012014  
 -1.51499e-7  -1.26041e-6  -1.26041e-6      0.0012014     0.00999052 

* standard errors of the estimated mean parameters

In [28]:
Bse

2×2 Array{Float64,2}:
 0.0425881  0.0484485
 0.0430919  0.0493475

* covariance matrix of the mean parameter estimates

In [29]:
Bcov

4×4 Array{Float64,2}:
  0.00181375   -1.96783e-5    0.000239868  -4.34611e-6 
 -1.96783e-5    0.00185691   -4.34611e-6    0.000242745
  0.000239868  -4.34611e-6    0.00234726   -5.73082e-6 
 -4.34611e-6    0.000242745  -5.73082e-6    0.00243518 

### Optimization algorithms 

Finding the MLE or REML of variance component models is a non-trivial nonlinear optimization problem. The main complications are the non-convexity of objective function and the positive semi-definiteness constraint of variane component parameters $\Sigma_1,\ldots,\Sigma_m$. In specific applications, users should try different algorithms with different starting points in order to find a better solution. Here are some tips for efficient computation.
In general the optimization algorithm needs to invert the $nd$ by $nd$ overall covariance matrix $\Omega = \Sigma_1 \otimes V_1 + \cdots + \Sigma_m \otimes V_m$ in each iteration. Inverting a matrix is an expensive operation with $O(n^3 d^3)$ floating operations. When there are only two varianec components ($m=2$), this tedious task can be avoided by taking one (generalized) eigendecomposion of $(V_1, V_2)$ and rotating data $(Y, X)$ by the eigen-vectors.

In [48]:
vcdatarot = TwoVarCompVariateRotate(vcdata)
fieldnames(typeof(vcdatarot))

(:Yrot, :Xrot, :eigval, :eigvec, :logdetV2)

Two optimization algorithms are implemented: [Fisher scoring](https://books.google.com/books?id=QYqeYTftPNwC&lpg=PP1&pg=PA142#v=onepage&q&f=false) (`mle_fs!`) and the [minorization-maximization (MM) algorithm](http://hua-zhou.github.io/media/pdf/ZhouHuZhouLange19VCMM.pdf) (`mle_mm!`). Both take the rotated data as input. These two functions give finer control of the optimization algorithms. Generally speaking, MM algorithm is more stable while Fisher scoring (if it converges) yields more accurate answer.

In [31]:
vcmodel_mm = deepcopy(vcmodel)
@time mle_mm!(vcmodel_mm, vcdatarot; maxiter=10000, funtol=1e-8, verbose = true);


     MM Algorithm
  Iter      Objective  
--------  -------------
       0  -6.253551e+03
       1  -3.881454e+03
       2  -3.853179e+03
       3  -3.846525e+03
       4  -3.844906e+03
       5  -3.844506e+03
       6  -3.844406e+03
       7  -3.844381e+03
       8  -3.844375e+03
       9  -3.844374e+03
      10  -3.844373e+03

  0.060365 seconds (19.34 k allocations: 1.274 MiB)


In [32]:
# MM estimates
vcmodel_mm.B

2×2 Array{Float64,2}:
 1.092     1.04727
 0.955346  1.01632

In [33]:
# MM estimates
vcmodel_mm.Σ

([0.3806371846890906 -0.30546523211690463; -0.30546523211690463 4.519380324112873], [1.8400909904775102 0.26556859919650166; 0.26556859919650166 2.172752980034015])

Fisher scoring (`mle_fs!`) uses either [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) (keyword `solver=:Ipopt`) or [KNITRO.jl](https://github.com/JuliaOpt/KNITRO.jl) (keyword `solver=:Knitro`) as the backend solver. Ipopt is open source and installation of [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) package alone is sufficient. Knitro is a commercial software and users need to follow instructions at [KNITRO.jl](https://github.com/JuliaOpt/KNITRO.jl) for proper functioning.

In [34]:
# Fisher scoring using Ipopt
vcmodel_ipopt = deepcopy(vcmodel)
@time mle_fs!(vcmodel_ipopt, vcdatarot; solver=:Ipopt, maxiter=1000, verbose=true);

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [35]:
# Ipopt estimates
vcmodel_ipopt.B

2×2 Array{Float64,2}:
 1.092     1.04727
 0.955346  1.01632

In [36]:
# Ipopt estimates
vcmodel_ipopt.Σ

([0.3805517356051958 -0.3055935197839397; -0.3055935197839397 4.521059353384276], [1.8400802765380622 0.2653850252233819; 0.2653850252233819 2.172870198791716])

In [37]:
# Fisher scoring using Knitro
vcmodel_knitro = deepcopy(vcmodel)
@time mle_fs!(vcmodel_knitro, vcdatarot; solver=:Knitro, maxiter=1000, verbose=true);

UndefVarError: UndefVarError: KnitroSolver not defined

### Starting point

Here are a few strategies for successful optimization.

* For $d>1$ (multivariate response), initialize $B, \Sigma$ from univariate estimates.
* Use REML estimate as starting point for MLE.
* When there are only $m=2$ variance components, pre-compute `TwoVarCompVariateRotate` and use it for optimization.


## Constrained estimation of B

Many applications invoke constraints on the mean parameters B. For demonstration, we enforce `B[1,1]=B[1,2]` and all entries of B are within [0, 2].

In [38]:
# set up constraints on B
vcmodel_constr = deepcopy(vcmodel)
vcmodel_constr.A = [1.0 0.0 -1.0 0.0]
vcmodel_constr.sense = '='
vcmodel_constr.b = 0.0
vcmodel_constr.lb = 0.0
vcmodel_constr.ub = 2.0
vcmodel_constr

VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.0 0.0; 0.0 0.0], ([1.0 0.0; 0.0 1.0], [1.0 0.0; 0.0 1.0]), [1.0 0.0 -1.0 0.0], '=', 0.0, 0.0, 2.0)

In [39]:
# MM algorithm for constrained estimation of B
@time mle_mm!(vcmodel_constr, vcdatarot; maxiter=10000, funtol=1e-8, verbose = true);


     MM Algorithm
  Iter      Objective  
--------  -------------
       0  -6.253551e+03
       1  -3.881820e+03
       2  -3.853477e+03
       3  -3.846807e+03
       4  -3.845184e+03
       5  -3.844783e+03
       6  -3.844683e+03
       7  -3.844658e+03
       8  -3.844652e+03
       9  -3.844650e+03
      10  -3.844650e+03

  0.191956 seconds (157.81 k allocations: 8.210 MiB)


In [49]:
fieldnames(typeof(vcmodel_constr))

(:B, :Σ, :A, :sense, :b, :lb, :ub)

In [43]:
vcmodel_constr.B

2×2 Array{Float64,2}:
 1.07177   1.07177
 0.955683  1.01591

In [44]:
vcmodel_constr.Σ

([0.38062446987069126 -0.30549750554026933; -0.30549750554026933 4.519477725102305], [1.8405108260586278 0.2650648215816033; 0.2650648215816033 2.1733573050720496])

Now let's try Fisher scoring.

In [45]:
# Fisher scoring using Ipopt for constrained estimation of B
vcmodel_constr = deepcopy(vcmodel)
vcmodel_constr.A = [1.0 0.0 -1.0 0.0]
vcmodel_constr.sense = '='
vcmodel_constr.b = 0.0
vcmodel_constr.lb = 0.0
vcmodel_constr.ub = 2.0
vcmodel_constr
@time mle_fs!(vcmodel_constr, vcdatarot; solver=:Ipopt, maxiter=1000, verbose=true);

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [46]:
vcmodel_constr.B

2×2 Array{Float64,2}:
 1.07177   1.07177
 0.955683  1.01591

In [47]:
vcmodel_constr.Σ

([0.38053902231656933 -0.30562579795954326; -0.30562579795954326 4.521156860574265], [1.840500109410487 0.2648809570723453; 0.2648809570723453 2.173475215913481])

## Heritability Analysis 